In [1]:
import os

In [24]:
path_to_scores = "runs_automatic/"
overleaf_output = []
for file in os.listdir(path_to_scores):
    if file[-6:] == "scores":
        scores = []
        with open(path_to_scores + file, "r") as f:
            for line in f:
                scores.append(line.strip("\n").split())
        split_scores = file.split(".")
        disk = split_scores[1]
        queries = split_scores[2]
        source = split_scores[3]
        weight = split_scores[4]
        if split_scores[5] in ["1","5"]:
            weight += "." + split_scores[5]
            method = split_scores[6]
        else:
            method = split_scores[5]

        if "_" in weight:
            weight = "\_".join(weight.split("_"))
        map =  scores[0][2]
        p30 = scores[1][2]
        recall = scores[2][2]

        overleaf_output.append([disk,queries,source,weight,method,map,p30,recall])


sorted_overleaf_output = sorted(overleaf_output, reverse=True, key=lambda x: float(x[-1]))
sorted_overleaf_output = ["Disk & Queries & Expansion Source & Weighting & Method & MAP & P@30 & R@1000 \\\\"] + ["\\hline"] + [" & ".join(x) + "\\\\" for x in sorted_overleaf_output]
for line in sorted_overleaf_output:
    print(line)

Disk & Queries & Expansion Source & Weighting & Method & MAP & P@30 & R@1000 \\
\hline
disk12 & 51-100 & wordnet & contextual\_term & bm25 & 0.2287 & 0.4520 & 0.4922\\
disk12 & 51-100 & wordnet & 1 & bm25 & 0.2277 & 0.4540 & 0.4895\\
disk12 & 51-100 & wordnet & 0 & bm25s & 0.2276 & 0.4540 & 0.4895\\
disk12 & 51-100 & wordnet & 0.1 & bm25s & 0.2276 & 0.4540 & 0.4895\\
disk12 & 51-100 & optimal & 1 & bm25 & 0.2277 & 0.4540 & 0.4895\\
disk12 & 51-100 & wordnet & contextual\_term & bm25s & 0.2270 & 0.4527 & 0.4893\\
disk12 & 51-100 & wordnet & contextual\_mean & bm25s & 0.2275 & 0.4540 & 0.4887\\
disk12 & 51-100 & wordnet & 0.5 & bm25s & 0.2271 & 0.4540 & 0.4884\\
disk12 & 51-100 & wordnet & wordnet\_wup & bm25s & 0.2274 & 0.4540 & 0.4878\\
disk12 & 51-100 & wordnet & wordnet\_path & bm25s & 0.2274 & 0.4540 & 0.4878\\
disk12 & 51-100 & optimal & 0.5 & bm25 & 0.2241 & 0.4527 & 0.4876\\
disk12 & 51-100 & optimal & 0.1 & bm25 & 0.2182 & 0.4433 & 0.4816\\
disk12 & 51-100 & wordnet & 0.5 & bm25